In [2]:
!pip install datamodel-code-generator

  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.7.2
    Uninstalling jsonschema-4.7.2:
      Successfully uninstalled jsonschema-4.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
warlock 2.0.1 requires jsonschema<5,>=4, but you have jsonschema 3.2.0 which is incompatible.


In [3]:
!datamodel-codegen --input gha-json-schema.json --output models.py --allow-population-by-field-name

In [ ]:
import requests 

def download_schema(url, filename):
    with open(filename, "wb") as f:
        content = requests.get(url).content
        f.write(content)

In [ ]:
url = "https://json.schemastore.org/github-workflow.json"
filename = "gha-json-schema.json"

# download_schema(url, filename)

In [1]:
import models

 This is a PoC using the Pydantic models generated from `datamodel-codegen` to recreate the `qhub-linter.yaml` GitHub Action. It was initially created from [Pydantic models that were hand-coded](https://github.com/Quansight/qhub/blob/4f2802524ad28ea07597f7ad8dd18abe83973020/qhub/provider/cicd/github.py#L131).

In [2]:
def get_json(model):
    return model.json(
        indent=2,
        by_alias=True,
        exclude_unset=True,
        exclude_defaults=True,
    )

In [3]:
def pip_install_qhub(version):
    return f"pip install qhub=={version}"

In [4]:
qhub_version = "0.4.3"
PYTHON_VERSION = "3.10"

In [5]:
step1 = models.Step(
    name="Checkout Image",
    uses="actions/checkout@master",
    with_={"token": "${{ secrets.REPOSITORY_ACCESS_TOKEN }}"},
)

In [6]:
step2 = models.Step(
    name="Set up Python",
    uses="actions/setup-python@v2",
    with_={"python-version": PYTHON_VERSION},
)

In [7]:
step3 = models.Step(
    name="Install QHub", 
    run=pip_install_qhub(qhub_version)
)

In [8]:
step4_envs = {
    "PR_NUMBER": "${{ github.event.number }}",
    "REPO_NAME": "${{ github.repository }}",
    "GITHUB_TOKEN": "${{ secrets.REPOSITORY_ACCESS_TOKEN }}",
}

step4 = models.Step(
    name="QHub Lintify",
    run="qhub validate --config qhub-config.yaml --enable-commenting",
    env=step4_envs,
)

In [9]:
jobs = models.NormalJob(
    name="qhub",
    runs_on="ubuntu-latest",
    steps=[step1, step2, step3, step4],
)

In [10]:
refs = models.Ref1(
    branches=["main"],
    paths=["qhub-config.yaml"]
)

In [11]:
on = models.OnItem(
    pull_request=refs
)

In [12]:
qhub_linter = models.Model(
    name="qhub linter",
    on=on,
    jobs={"qhub-validate": jobs},
)

In [13]:
print(get_json(qhub_linter))

{
  "name": "qhub linter",
  "on": {
    "pull_request": {
      "branches": [
        "main"
      ],
      "paths": [
        "qhub-config.yaml"
      ]
    }
  },
  "jobs": {
    "qhub-validate": {
      "name": "qhub",
      "runs-on": "ubuntu-latest",
      "steps": [
        {
          "name": "Checkout Image",
          "uses": "actions/checkout@master",
          "with": {
            "token": "${{ secrets.REPOSITORY_ACCESS_TOKEN }}"
          }
        },
        {
          "name": "Set up Python",
          "uses": "actions/setup-python@v2",
          "with": {
            "python-version": "3.10"
          }
        },
        {
          "name": "Install QHub",
          "run": "pip install qhub==0.4.3"
        },
        {
          "name": "QHub Lintify",
          "run": "qhub validate --config qhub-config.yaml --enable-commenting",
          "env": {
            "PR_NUMBER": "${{ github.event.number }}",
            "REPO_NAME": "${{ github.repository }}",
           

In [14]:
fn = "qhub-linter-new.yaml"

with open(fn, "w") as f:
    f.write(get_json(qhub_linter))